# STEP 2 - COMBINE ALL RISK FACTORS

##### Created by - Sanjana Tule 
##### Date - 14/08/2021



> Choose pickles files to combine the risk factors into 1 graph

In [1]:
# !pip install osmnx
# !apt install python3-rtree

In [36]:
import osmnx as ox
import networkx as nx
import numpy as np
import geopandas

In [4]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
# read all pickles

# distance pickle
graph_distance = nx.read_gpickle('/content/drive/My Drive/omdena/earthquake/1_1_northridge_graph_distance_risk_added.pickle')
nodes_distance,edges_distance = ox.graph_to_gdfs(graph_distance)
buildings = nx.read_gpickle('/content/drive/My Drive/omdena/earthquake/northridge_footprint.pickle')
gdf = nx.read_gpickle('/content/drive/My Drive/omdena/earthquake/northridge_gdf.pickle')

# building density pickle
density_pickle = nx.read_gpickle('/content/drive/My Drive/omdena/earthquake/1_2_northridge_graph_building_density_risk_addded.pickle')
nodes_bd,edges_bd = ox.graph_to_gdfs(density_pickle)

# other pickles add here

In [62]:
# combine pickles

#remove extra columns
edges_distance = edges_distance.drop(columns = ['sampling_counts','max','min','median','x_y','lon_lat','mean_scaled','mean','median_scaled','mean_scaled_inversed','median_scaled_inversed'])
edges_bd = edges_bd.drop(columns=['name','highway','maxspeed','oneway','length','lanes','access','ref','service','bridge','width','tunnel'])
edges_bd = edges_bd.rename(columns={"Building": "building_risk_score"})
edges_distance.head()

# combine on geospatial attributes
combined_risk_edges = geopandas.sjoin(edges_distance, edges_bd, how="inner", op='intersects')
# drop duplicate columns
combined_risk_edges = combined_risk_edges.drop(columns=['index_right0','index_right1','index_right2','osmid_right'])
combined_risk_edges.head()

osmid_left           name  ... tunnel building_risk_score
u          v          key                            ...                           
29409715   6403348096 0     15240187   Chase Street  ...    NaN               197.0
           6403348105 0    400864518  Corbin Avenue  ...    NaN               197.0
6403348096 29409715   0     15240187   Chase Street  ...    NaN               197.0
           4269165902 0     15240187   Chase Street  ...    NaN               197.0
           6403348097 0    683632624            NaN  ...    NaN               197.0

[5 rows x 16 columns]

In [64]:
# Create a new graph and save in pickle file
combined_risk_graph = ox.graph_from_gdfs(nodes, combined_risk_edges)
nx.write_gpickle(combined_risk_graph, '/content/drive/My Drive/omdena/earthquake/1_northridge_graph_all_risk_added.pickle')
